# 📈 ASTP – Projeto Final
## Previsão de atrasos e cancelamentos de voos nos Estados Unidos, com base em fatores meteorológicos

### 👥 Grupo XX
- **Pedro Fonte Santa**, nº 105306  
- **Rafael Alexandre Dias Andorinha**, nº 131000  
- **Vasco Marques**, nº [Número de Aluno]  

---

📅 **Data de entrega:** 19 de maio de 2025  

**Objetivo deste Notebook**: Este notebook corresponde à limpeza do dataset do **Kaggle** de Eventos Climáticos nos EUA.

---

### 🗂️ Datasets:
- [Previsão de estados de Voos (Kaggle)](https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022/data?select=raw)  
- [Eventos Climáticos nos EUA (Kaggle)](https://www.kaggle.com/datasets/sobhanmoosavi/us-weather-events/data)


Importamos o dataset meteorológico, aplicamos os seguintes filtros:

- Apenas eventos entre **2018 e 2022**
- Apenas saverity **Light** e **Severe**
- Remoção da saverity **Other**

In [1]:
import pandas as pd

In [ ]:
caminho_dados = "/Users/pedrofs/Library/CloudStorage/OneDrive-ISCTE-IUL/Mestrado/2ªSem/ASTP/weather-events-2016-2022.csv"

# Leitura
df = pd.read_csv(caminho_dados)

df.groupby(['Type', 'Severity']).size().reset_index(name='Contagem')


,Type,Severity,Contagem
0,Cold,Severe,232192
1,Fog,Moderate,578683
2,Fog,Severe,1435507
3,Hail,Other,2927
4,Precipitation,UNK,157186
5,Rain,Heavy,180098
6,Rain,Light,4159503
7,Rain,Moderate,661193
8,Snow,Heavy,39053
9,Snow,Light,990283


In [12]:
# Converter datas
df['StartTime(UTC)'] = pd.to_datetime(df['StartTime(UTC)'])
df['Date'] = pd.to_datetime(df['StartTime(UTC)'].dt.date)

# Filtrar datas entre 2018 e 2022
df = df[(df['Date'] >= pd.to_datetime("2018-01-01")) & (df['Date'] <= pd.to_datetime("2022-12-31"))]

# Remover severidades indesejadas: 'Other' e 'UNK'
df = df[~df['Severity'].isin(['Other', 'UNK'])]

# Verificação
df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,Date
965,W-966,Rain,Light,2018-01-07 03:36:00,2018-01-07 03:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2018-01-07
966,W-967,Snow,Light,2018-01-07 03:54:00,2018-01-07 07:12:00,0.02,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2018-01-07
967,W-968,Snow,Light,2018-01-11 03:36:00,2018-01-11 03:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2018-01-11
968,W-969,Snow,Light,2018-01-12 13:54:00,2018-01-12 15:12:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2018-01-12
969,W-970,Snow,Light,2018-01-12 15:54:00,2018-01-12 16:36:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2018-01-12


In [13]:
df.tail()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,Date
8627176,W-9091890,Snow,Light,2022-12-21 23:00:00,2022-12-21 23:42:00,0.01,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-21
8627177,W-9091891,Snow,Moderate,2022-12-21 23:42:00,2022-12-21 23:53:00,0.04,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-21
8627178,W-9091892,Cold,Severe,2022-12-21 23:53:00,2022-12-24 02:53:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-21
8627179,W-9091893,Cold,Severe,2022-12-24 03:53:00,2022-12-24 07:53:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-24
8627180,W-9091894,Cold,Severe,2022-12-24 09:53:00,2022-12-24 11:53:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-24


### Agrupamento Diário

>Criamos uma tabela com o número de eventos meteorológicos por dia e tipo, somando todas as severiy (**Light + Severe**) num mesmo tipo.

In [14]:
# Agrupar por data e tipo
eventos_diarios = df.groupby(['Date', 'Type']).size().reset_index(name='count')

# Pivot para ter os tipos como colunas
series_diarias = eventos_diarios.pivot(index='Date', columns='Type', values='count').fillna(0).astype(int)

# Garantir ordenação por data
series_diarias = series_diarias.sort_index()

# Ver
series_diarias.head()

Type,Cold,Fog,Rain,Snow,Storm
Date,,,,,
2018-01-01,444,651,314,716,17
2018-01-02,422,684,211,619,18
2018-01-03,99,543,543,1097,5
2018-01-04,189,787,406,1408,23
2018-01-05,298,844,544,923,16


In [15]:
# Ver
series_diarias.tail()

Type,Cold,Fog,Rain,Snow,Storm
Date,,,,,
2022-12-27,70,541,1491,601,54
2022-12-28,59,510,831,734,51
2022-12-29,65,678,904,744,12
2022-12-30,69,1113,2725,812,9
2022-12-31,82,2839,3350,536,9


In [16]:
# Guardar no local desejado
series_diarias.to_csv("../../data/weather-events-2018-2022_Final.csv", index=True)
print("Dataset meteorológico diário salvo com sucesso!")

Dataset meteorológico diário salvo com sucesso!


### Para o relatorio


O pré-processamento envolveu os seguintes passos:
- Filtro temporal para manter apenas os eventos entre **2018 e 2022**, alinhando com o período das séries de atrasos e cancelamentos.
- Conversão do campo de início (`StartTime`) para o formato de data (`Date`), de forma a permitir a agregação diária.
- Remoção de eventos com severidade classificada como **"Other"** ou **"UNK"**, por representarem categorias pouco informativas ou indefinidas.
- Agrupamento do número de eventos por **tipo (ex: Rain, Fog, Snow)** e por **dia**, somando todas as ocorrências em cada data.

O resultado final é um dataset com 5 séries temporais diárias:
**Cold, Fog, Rain, Snow e Storm**, representando a frequência de cada tipo de evento meteorológico entre 2018 e 2022. Este dataset será usado nas análises de causalidade para investigar potenciais relações com atrasos e cancelamentos.